In [2]:
import pymongo
import pandas as pd


db = pymongo.MongoClient("mongodb://127.0.0.1:27017")["radar"]
col = db["package_repository_url"]
baseline_results = pd.DataFrame(col.find({}, projection={"_id": 0}))
print(len(baseline_results))
baseline_results.head()

4227425


,name,version,OSSGadget,Warehouse,Libraries.io,py2src_ossgadget,py2src_badge,py2src_metadata,py2src_readthedocs,py2src_statistics,py2src_final,py2src_homepage
0,sm2kg,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,brkout,0.6,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,NaN,https://github.com/thealphadollar/brkout,NaN,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout
2,pynamd,0.0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mongogrant,0.2.1,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
4,mongogrant,0.1.4,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant


In [3]:
print(
    f"Total: {len(baseline_results)} releases, {len(baseline_results['name'].unique())} packages"
)
for column in ["OSSGadget", "Warehouse", "Libraries.io", "py2src_final"]:
    releases = baseline_results[baseline_results[column].notna()][
        ["name", "version", column]
    ]
    num_packages = len(releases["name"].unique())
    print(f"{column}: {len(releases)} releases, {num_packages} packages")

Total: 4227425 releases, 423725 packages
OSSGadget: 3072108 releases, 290122 packages
Warehouse: 3068148 releases, 289919 packages
Libraries.io: 3161243 releases, 299153 packages
py2src_final: 3189944 releases, 297443 packages


In [4]:
librariesio_results = baseline_results[["name", "version", "Libraries.io"]].copy()
librariesio_results.loc[
    librariesio_results["Libraries.io"].notna(), "platform"
] = librariesio_results[librariesio_results["Libraries.io"].notna()][
    "Libraries.io"
].apply(
    lambda x: x.split("/")[2]
)
librariesio_results.head()

,name,version,Libraries.io,platform
0,sm2kg,0.1,NaN,NaN
1,brkout,0.6,https://github.com/thealphadollar/brkout,github.com
2,pynamd,0.0.1,NaN,NaN
3,mongogrant,0.2.1,https://github.com/materialsproject/mongogrant,github.com
4,mongogrant,0.1.4,https://github.com/materialsproject/mongogrant,github.com


In [5]:
platform_dist = (
    librariesio_results["platform"]
    .value_counts(dropna=False)
    .to_frame(name="num_releases")
)
platform_dist["percentage_releases"] = (
    platform_dist["num_releases"] / platform_dist["num_releases"].sum() * 100
)
platform_dist["num_packages"] = librariesio_results.groupby("platform", dropna=False)[
    "name"
].apply(lambda x: len(set(x)))
platform_dist["percentage_packages"] = (
    platform_dist["num_packages"] / platform_dist["num_packages"].sum() * 100
)
platform_dist

,num_releases,percentage_releases,num_packages,percentage_packages
github.com,3032984,71.745424,286539,63.490224
NaN,1066182,25.220601,149798,33.191672
gitlab.com,68670,1.624393,7533,1.669134
bitbucket.org,53668,1.269520,6451,1.429388
sourceforge.net,5918,0.139991,989,0.219139
svn.apache.org,2,0.000047,1,0.000222
git-wip-us.apache.org,1,0.000024,1,0.000222


In [6]:
print(
    len(
        baseline_results[
            baseline_results["OSSGadget"] == baseline_results["Libraries.io"]
        ]
    )
)
print(
    len(
        baseline_results[
            baseline_results["OSSGadget"] != baseline_results["Libraries.io"]
        ]
    )
)

3005599
1221826


In [7]:
ossgadget_warehouse_diff = baseline_results[
    (baseline_results["OSSGadget"].notna() | baseline_results["Warehouse"].notna())
    & (baseline_results["OSSGadget"] != baseline_results["Warehouse"])
][["name", "version", "OSSGadget", "Warehouse"]]
print(len(ossgadget_warehouse_diff))
ossgadget_warehouse_diff.head()

6862


,name,version,OSSGadget,Warehouse
4442,sbq,0.1.0,https://github.com/cjfuller/sbq,NaN
15049,zmei-utils,0.1.14,NaN,https://github.com/negative space ou/zmei
15050,zmei-utils,0.1.15,NaN,https://github.com/negative space ou/zmei
15051,zmei-utils,0.1.9,NaN,https://github.com/negative space ou/zmei
15052,zmei-utils,0.1.8,NaN,https://github.com/negative space ou/zmei


In [8]:
print(
    len(
        ossgadget_warehouse_diff[
            ossgadget_warehouse_diff["OSSGadget"].notna()
            & ossgadget_warehouse_diff["Warehouse"].isna()
        ]
    )
)
print(
    len(
        ossgadget_warehouse_diff[
            ossgadget_warehouse_diff["OSSGadget"].isna()
            & ossgadget_warehouse_diff["Warehouse"].notna()
        ]
    )
)
print(
    len(
        ossgadget_warehouse_diff[
            ossgadget_warehouse_diff["OSSGadget"].notna()
            & ossgadget_warehouse_diff["Warehouse"].notna()
        ]
    )
)

4579
619
1664


## Warehouse (urlparse) vs. OSSGadget (regex)

1. non-standard URL, e.g., "https://github.com/fm-17/poglink,", "github.com/andrewlorente/catsnap", "https://3044d22ab9179f52ecb34567f62c8b9819f0333d@github.com/ailever/ailever", 
2. Random URL access, e.g., ignition-api-stubs-8.1.18a0
3. Reserved github accounts, e.g., meerschaum-1.4.11

In [9]:
redirect_dist = baseline_results[
    (
        baseline_results["OSSGadget"].notna()
        | baseline_results["py2src_ossgadget"].notna()
    )
    & (baseline_results["OSSGadget"] != baseline_results["py2src_ossgadget"])
][["name", "version", "OSSGadget", "py2src_ossgadget"]]
print(
    len(redirect_dist),
    len(redirect_dist) / len(baseline_results[baseline_results["OSSGadget"].notna()]),
)
print("404 releases:", len(redirect_dist[redirect_dist["py2src_ossgadget"].isna()]))
print(
    "redirect releases:",
    len(
        redirect_dist[
            redirect_dist["py2src_ossgadget"].notna()
            & (redirect_dist["py2src_ossgadget"] != redirect_dist["OSSGadget"])
        ]
    ),
)

632166 0.2057759688135964
404 releases: 319215
redirect releases: 312951


In [10]:
py2src_columns = [
    "py2src_ossgadget",
    "py2src_badge",
    "py2src_metadata",
    "py2src_readthedocs",
    "py2src_statistics",
    "py2src_homepage",
]
baseline_results[~baseline_results[py2src_columns].isna().all(axis=1)]

,name,version,OSSGadget,Warehouse,Libraries.io,py2src_ossgadget,py2src_badge,py2src_metadata,py2src_readthedocs,py2src_statistics,py2src_final,py2src_homepage
1,brkout,0.6,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,NaN,https://github.com/thealphadollar/brkout,NaN,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout
3,mongogrant,0.2.1,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
4,mongogrant,0.1.4,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
5,mongogrant,0.2.0,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
6,mongogrant,0.1.2,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
...,...,...,...,...,...,...,...,...,...,...,...,...
4227419,brainpyfuck,0.0.4,https://github.com/forever-codingnoob/brainfuck,https://github.com/forever-codingnoob/brainfuck,https://github.com/forever-codingnoob/brainfuck,https://github.com/forever-codingnoob/brainfuck,NaN,https://github.com/forever-codingnoob/brainfuck,NaN,https://github.com/forever-codingnoob/brainfuck,https://github.com/forever-codingnoob/brainfuck,NaN
4227421,mitmproxy-escher,2.0.0,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,NaN,https://github.com/knagy/mitmproxy-escher,NaN,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher
4227422,mitmproxy-escher,1.0.0,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,NaN,https://github.com/knagy/mitmproxy-escher,NaN,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher,https://github.com/knagy/mitmproxy-escher
4227423,py-oathtool,1.1.0,https://github.com/matalo33/py_oathtool,https://github.com/matalo33/py_oathtool,https://github.com/matalo33/py_oathtool,https://github.com/matalo33/py_oathtool,NaN,https://github.com/matalo33/py_oathtool,NaN,https://github.com/matalo33/py_oathtool,https://github.com/matalo33/py_oathtool,NaN


In [26]:
py2src_results = baseline_results[baseline_results["py2src_final"].notna()][
    [
        "name",
        "version",
        "py2src_ossgadget",
        "py2src_badge",
        "py2src_metadata",
        "py2src_readthedocs",
        "py2src_statistics",
        "py2src_homepage",
    ]
].copy()
print(len(py2src_results))
py2src_results.head()

3189944


,name,version,py2src_ossgadget,py2src_badge,py2src_metadata,py2src_readthedocs,py2src_statistics,py2src_homepage
1,brkout,0.6,https://github.com/thealphadollar/brkout,NaN,https://github.com/thealphadollar/brkout,NaN,https://github.com/thealphadollar/brkout,https://github.com/thealphadollar/brkout
3,mongogrant,0.2.1,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
4,mongogrant,0.1.4,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
5,mongogrant,0.2.0,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant
6,mongogrant,0.1.2,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,NaN,https://github.com/materialsproject/mongogrant,https://github.com/materialsproject/mongogrant


In [12]:
ossgadget_releases = py2src_results[py2src_results["py2src_ossgadget"].notna()]
ossgadget_packages = set(ossgadget_releases["name"].unique())
print(
    f"OSSGadget: {len(ossgadget_releases)} releases, {len(ossgadget_packages)} packages"
)
for column in ["py2src_badge", "py2src_readthedocs", "py2src_homepage"]:
    releases = py2src_results[
        py2src_results["py2src_ossgadget"].isna() & py2src_results[column].notna()
    ]
    packages = set(releases["name"].unique()) - ossgadget_packages
    print(f"{column}: {len(releases)} releases, {len(packages)} packages")

OSSGadget: 2752893 releases, 257573 packages
py2src_badge: 73563 releases, 5040 packages
py2src_readthedocs: 58207 releases, 4349 packages
py2src_homepage: 29249 releases, 1191 packages


In [27]:
def same(x):
    urls = [x[c] for c in py2src_columns if pd.notna(x[c])]
    if len(set(urls)) == 1:
        return True
    return False


py2src_results["same"] = py2src_results.apply(same, axis=1)

In [29]:
len(py2src_results[py2src_results["same"] == False])

443322

## Some reasons for the differences among different measures

1. redirection
2. wrong readthedocs url
3. reserved names